In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
 print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d jangedoo/utkface-new

In [ ]:
!unzip utkface-new.zip

In [ ]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [ ]:
age = []
gender = []
img_path = []
for file in os.listdir(folder_path):
    age.append(int(file.split('_')[0]))
    gender.append(int(file.split('_')[1]))
    img_path.append(file)

In [ ]:
len(age)

In [ ]:
df = pd.DataFrame({'age':age ,'gender':gender ,'img_path':img_path})

In [ ]:
df.head()

In [ ]:
train_df = df.sample(frac=1 ,random_state=0).iloc[:20000]
test_df = df.sample(frac=1 ,random_state=0).iloc[20000:]

In [ ]:
len(train_df) ,len(test_df)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255 ,
                                   rotation_range=30 ,
                                   width_shift_range=0.2 ,
                                   height_shift_range=0.2 ,
                                   shear_range=0.2 ,
                                   zoom_range=0.2 ,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(train_df ,
                                                    directory=folder_path ,
                                                    x_col='img_path' ,
                                                    y_col=['age','gender'] ,
                                                    target_size=(200,200) ,
                                                    class_mode='multi_output')
test_generator = test_datagen.flow_from_dataframe(test_df ,
                                                    directory=folder_path ,
                                                    x_col='img_path' ,
                                                    y_col=['age','gender'] ,
                                                    target_size=(200,200) ,
                                                    class_mode='multi_output')

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model

In [ ]:
vggnet = VGG16(include_top=False ,input_shape=(200,200,3))

In [ ]:
vggnet.trainable = False

output = vggnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512 ,activation='relu')(flatten)
dense2 = Dense(512 ,activation='relu')(flatten)

dense3 = Dense(512 ,activation='relu')(dense1)
dense4 = Dense(512 ,activation='relu')(dense2)

output1 = Dense(1 ,activation='linear' ,name='age')(dense3)
output2 = Dense(1 ,activation='sigmoid' ,name='gender')(dense4)

In [ ]:
model = Model(inputs=vggnet.input ,outputs=[output1 ,output2])

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model)

In [ ]:
model.compile(optimizer='adam' ,loss={'age':'mae' ,'gender':'binary_crossentropy'} ,metrics={'age':'mae' ,'gender':'accuracy'} ,loss_weights={'age':1 ,"gender":99} )

In [ ]:
model.fit(train_generator ,batch_size=32 ,epochs=10 ,validation_data=test_generator)

In [ ]:
import pickle
with open('model.pickle','wb') as file:
  pickle.dump(model ,file)

In [ ]:
prediction = model.predict(test_generator)

age_predictions = prediction[0]
gender_predictions = prediction[1]


In [ ]:
gender_predictions[:5]

In [ ]:
age_predictions[:5]

In [ ]:
test_generator.image_shape

In [ ]:
from PIL import Image

image_path = '/content/utkface_aligned_cropped/UTKFace/10_0_0_20170110220235233.jpg.chip.jpg'
image = Image.open(image_path)

image = image.resize((200, 200))
image = np.array(image) / 255.0
image = np.expand_dims(image, axis=0)

predictions = model.predict(image)

age_prediction = predictions[0][0]
gender_prediction = predictions[1][0]

# if gender_prediction < 5:
#   gender = 'male'
# else:
#   gender = 'female'

print(f"Predicted Age: {age_prediction}")
print(f"Predicted Gender: {gender_prediction} ")